In [29]:
from sqlalchemy import create_engine, Table, Column, Integer, String, Numeric, MetaData, text
import pandas as pd

# === создаём локальную базу SQLite (файл появится в текущей папке) ===
engine = create_engine('sqlite:///nakladnyeDB.db', echo=False)
metadata = MetaData()

# === описание таблицы ===
nakladnye = Table(
    'nakladnye', metadata,
    Column('Накладная номер', Integer, primary_key=True),
    Column('Отправитель', String(100)),
    Column('Получатель', String(100)),
    Column('Плательщик', String(100)),
    Column('Сумма', Numeric(10, 2))
)

metadata.drop_all(engine, tables=[nakladnye])
metadata.create_all(engine)
print("Таблица 'nakladnye' создана в локальной базе.")

# === данные ===
data = [
    (1, 'МТС', 'МТС', 'Иванов И.И.', 1000),
    (2, 'Иванов И.И.', 'Иванов И.И.', 'МТС', 2000),
    (3, 'МТС', 'Иванов И.И.', 'МТС', 4500),
    (4, 'Иванов И.И.', 'МТС', 'МТС', 2000),
    (5, 'Андреев Р.', 'Иванов И.И.', 'Иванов И.И.', 3000)
]

df = pd.DataFrame(data, columns=['Накладная номер', 'Отправитель', 'Получатель', 'Плательщик', 'Сумма'])

# === записываем данные ===
df.to_sql('nakladnye', con=engine, if_exists='replace', index=False)
print("Данные успешно добавлены.")

# === проверяем ===
with engine.connect() as conn:
    text_sql = """SELECT *
                  FROM nakladnye"""
    result = conn.execute(text(text_sql))
    for row in result:
        print(row)

Таблица 'nakladnye' создана в локальной базе.
Данные успешно добавлены.
(1, 'МТС', 'МТС', 'Иванов И.И.', 1000)
(2, 'Иванов И.И.', 'Иванов И.И.', 'МТС', 2000)
(3, 'МТС', 'Иванов И.И.', 'МТС', 4500)
(4, 'Иванов И.И.', 'МТС', 'МТС', 2000)
(5, 'Андреев Р.', 'Иванов И.И.', 'Иванов И.И.', 3000)


In [30]:
with engine.connect() as conn:
    text_sql = """WITH sender AS (
                    SELECT [Накладная номер], [Отправитель] AS [Клиент], [Сумма]
                    FROM nakladnye
                    ),
                  recipient AS (
                    SELECT [Накладная номер], [Получатель] AS [Клиент], [Сумма]
                    FROM nakladnye
                    ),
                  payer AS (
                    SELECT [Накладная номер], [Плательщик] AS [Клиент], [Сумма]
                    FROM nakladnye
                    ),
                  total AS (
                    SELECT *
                    FROM sender
                    UNION ALL
                    SELECT *
                    FROM recipient
                    UNION ALL
                    SELECT *
                    FROM payer)
                  SELECT [Клиент], SUM([Сумма]) as [Сумма]
                  FROM (SELECT DISTINCT *
                        FROM total)
                  GROUP BY [Клиент]
                  ORDER BY SUM([Сумма]) DESC
                  """
    result = conn.execute(text(text_sql))
    for row in result:
        print(row)

('Иванов И.И.', 12500)
('МТС', 9500)
('Андреев Р.', 3000)
